In [1]:
import holoviews as hv; hv.extension('plotly', logo=False)
import panel as pn;     pn.extension('plotly', 'katex', 'mathjax')
import plotly.graph_objects as go
import numpy as np
import sympy as sp

<div style="float:center;width:100%;text-align: center;">
    <strong style="height:60px;color:darkred;font-size:40px;">Decomposition of Vectors Using the Normal Equation</strong><br>
</div>

# 1. Drawing Routines

What happens when there are free variables in the normal equation $\mathbf{A^t A x = A^t b}$, i.e., when $A$ is **not full column rank?**<br>
$\qquad$ we will investigate a case where the fundamental spaces of $A$ consist of lines and planes.

First, develop some routines using Plotly to draw 3D vectors and planes.
* obtain traces for vector and planes
* create a figure with these traces
* display the figure using pn.pane.Plotly()

In [2]:
def create_fig( traces,x_range=(-10, 10), y_range=(-10, 10), z_range=(-10,10), camera_position=None, width=700 ):
    # Create the figure
    fig = go.Figure(data=traces)

    # Set the layout of the figure, including the camera position if specified
    camera = dict(eye=dict(x=-1.25, y=1.25, z=1.25))  # Default camera position
    if camera_position:
        camera['eye'] = camera_position

    fig.update_layout(scene=dict( xaxis=dict(nticks=4, range=x_range),
                                  yaxis=dict(nticks=4, range=y_range),
                                  zaxis=dict(nticks=4, range=z_range),
                                 ),
                      width=width,
                      margin=dict(r=2, l=1, b=1, t=1),
                      scene_camera=camera,
                      showlegend=False,
                     )
    return fig

In [3]:
def convert_2D_to_3D( v ):
    if len(v) == 2: return np.array( [v[0], v[1], 0])
    return np.array(v)

def is_number_not_iterable(variable):
    from collections.abc import Iterable
    import numbers
    # Check if the variable is a number
    if isinstance(variable, numbers.Number):
        # Check if the variable is not an iterable
        if not isinstance(variable, Iterable):
            return True
    return False

def line_segment_trace(start, end, color='blue', width=6):
    '''draw a line segment from start_point to end_point of given width and color (converts to 3D)'''
    start_point = convert_2D_to_3D( start )
    end_point   = convert_2D_to_3D( end )

    # Extract the coordinates of the start and end points
    x_coords = [start_point[0], end_point[0]]
    y_coords = [start_point[1], end_point[1]]
    z_coords = [start_point[2], end_point[2]]

    # Create the 3D line segment
    line = go.Scatter3d(
        x=x_coords,
        y=y_coords,
        z=z_coords,
        mode='lines',  # Define the mode as lines to draw a line segment
        line=dict(
            width=width,  # Set the line width
            color=color,  # Set the line color
        )
    )
    return [line]

def cone_trace(base_center, tip_location, base_radius, color='blue'):
    '''draw a cone from base_center to tip_location of given base_radius and color: note 3D vectors'''

    # Calculate the vector components from base to tip
    u = tip_location[0] - base_center[0]
    v = tip_location[1] - base_center[1]
    w = tip_location[2] - base_center[2]

    # Calculate the magnitude of the vector for normalization
    magnitude = np.sqrt(u**2 + v**2 + w**2)

    # Normalize the vector components to use as direction
    u /= magnitude
    v /= magnitude
    w /= magnitude

    # Create the cone trace
    cone = go.Cone(
        x = [base_center[0]], y = [base_center[1]],  z = [base_center[2]],
        u=[u], v=[v], w=[w],
        colorscale=[[0, color], [1, color]], showlegend=False, showscale=False,
        sizemode="absolute",
        sizeref=2*base_radius*magnitude,  # Adjust the cone size based on the base radius and vector magnitude
        anchor="center"                      # Anchor the cone at the base, tip or center
    )
    return [cone]

def vector_trace( start, end, arrow_length, arrow_size, color='blue'):
    '''draw a vector from start to end with an arrow of given length and base_radius "arrow_size" (converts to 3D)'''
    start = convert_2D_to_3D( start )
    end   = convert_2D_to_3D( end )


    dir_vec   = end - start
    base      = end - (arrow_length / np.linalg.norm( dir_vec )) * dir_vec
    cone      = cone_trace( base, end, arrow_size, color=color)
    line      = line_segment_trace( start, end-0.8*(end-base), color=color)
    return line+cone

def plane_from_basis_vectors_trace( vec1, vec2, x_range=(-10, 10), y_range=(-10, 10), plane_color='lightgray' ):
    '''draw a plane given 2D or 3D vectors vec1 and vec2
    Remark: will not draw a plane containing the z axis due to (x,y) meshgrid
    '''

    # Create a meshgrid for the plane
    #xx, yy        = np.meshgrid(range(x_range[0],x_range[1]+1), range(y_range[0],y_range[1]+1))
    xx, yy        = np.meshgrid( np.array(x_range), np.array(y_range))
    normal_vector = np.cross(vec1, vec2)
    f             = 0  if abs(normal_vector[2]) < 1e-10 else 1/normal_vector[2]
    z             = (-normal_vector[0] * xx - normal_vector[1] * yy) *f

    # Define the plane
    return [go.Surface(x=xx, y=yy, z=z, opacity=0.9, name='Plane', colorscale=[[0, plane_color], [1, plane_color]], showscale=False)]

# 2. The Normal Equation and the Decomposition of Vectors (3D Example)

## 2.1  Plane Defined from Basis Vectors together with a Third Vector

In [4]:
def projection_trace( vec_1, vec_2, vec_b, arrow_length, arrow_size, show_decomp=False,
                      x_range=(-10, 10), y_range=(-10, 10),
                      plane_color='lightgray', vec12_color='blue', vec_b_color='red', vec_decomp_color='darkred'):
    if is_number_not_iterable( arrow_length ): arrow_length = np.repeat( [arrow_length], 3 )
    if is_number_not_iterable( arrow_size   ): arrow_size   = np.repeat( [arrow_size  ], 3 )

    traces  = plane_from_basis_vectors_trace( vec_1, vec_2, x_range=x_range, y_range=y_range, plane_color=plane_color)
    traces += vector_trace( [0,0,0], vec_1, arrow_length[0], arrow_size[0], color=vec12_color)
    traces += vector_trace( [0,0,0], vec_2, arrow_length[1], arrow_size[1], color=vec12_color)
    traces += vector_trace( [0,0,0], vec_b, arrow_length[2], arrow_size[2], color=vec_b_color)
    if show_decomp:
        v1         = convert_2D_to_3D( vec_1 )
        v2         = convert_2D_to_3D( vec_2 )
        vb         = convert_2D_to_3D( vec_b )
        A          = np.stack( [v1, v2], axis=1)
        b_parallel = A@np.linalg.solve(A.T@A, A.T@vb)
        print( v1, v2, vb, b_parallel )
        traces += vector_trace( [0,0,0], b_parallel, arrow_length[0], arrow_size[0], color=vec_decomp_color)
        traces += vector_trace( b_parallel, vb,      arrow_length[2], arrow_size[2], color=vec_decomp_color)
    return traces

def projection_fig( v_1, v_2, v_b, arrow_length, arrow_size, show_decomp=False,
                      x_range=(-10, 10), y_range=(-10, 10), z_range=(-10,10),
                      camera_position = dict(x=-1.4, y=1., z=0.3), width=500,
                      plane_color='lightgray', vec12_color='blue', vec_b_color='red', vec_decomp_color='darkred'):
    return create_fig( projection_trace( v_1,v_2,v_b,arrow_length, arrow_size,
                                         x_range=x_range, y_range=y_range),
                       camera_position=camera_position, x_range=x_range, y_range=y_range, z_range=z_range, width=width )

In [5]:
# Two examples
camera_position = dict(x=-1.4, y=1., z=0.3)

# Figure 1:
v1 = np.array([1, 0, 0])
v2 = np.array([0, 1, 0])
v3 = np.array([0, 0, 1])          # vector not in the plane

# Figure 1:
u1 = np.array([1,   0.2, 0.1])
u2 = np.array([1,   1,   0.3])
u3 = np.array([0.5, 0.5, 1  ])    # vector not in the plane

fig1 = projection_fig( v1,v2,v3, [0.08,0.08,0.15], 1,   x_range=(-1.1,1.1), y_range=(-1.1,1.1), z_range=(-0.5,1.5), camera_position = camera_position, width=500)
fig2 = projection_fig( u1,u2,u3, 0.4, 0.3, x_range=(-1,2), y_range=(-1,2), z_range=(-1,1.2),   camera_position = camera_position, width=500)

pn.Column( pn.pane.Markdown("# Two Examples of a Plane Defined by Two Vectors, Together with a Third Vector"),
           pn.Row( pn.pane.Plotly(fig1), pn.pane.Plotly(fig2),  ))

Column
    [0] Markdown(str)
    [1] Row
        [0] Plotly(Figure)
        [1] Plotly(Figure)

This example shows 
* two **basis vectors** $\{ a_1, a_2 \}$ shown in blue. They are linearly independent, and therefore define **a plane**
* a **third vector** $b$ that is not in the plane (shown in red)<br>
The three vectors are linearly independent (a basis for $\mathbb{R}^3$.
* The plot on the left shows three mutually orthogonal unit length vectors (i,j,k)
* The plot on the right shows three arbitrary vectors forming a basis

## 2.2 A x = b with More than One Solution

* We know how to decompose the red vector into a vector in the plane + a vector perpendicular to the plane:<br>
$\qquad b = b_{_{//}} + b_{_\perp}$

$\qquad $ Solution: $\left\{ \begin{align}&  \text{  set} \qquad & A          & = ( a_1\; a_2 \; {\color{red}{\dots ?}} ) \\
                                 & \text{  solve}      & A^t  A x   & = A^t b \\
                                 & \text{  solve}      & b_{_{//}}  & = A x \\
                                 & \text{  solve}      & b_{_\perp} & = b - b_{_{//}} \\
                                 \end{align}\right.$

**What if the normal equation has an infinite number of solutions?** $x = x_p + x_h \Rightarrow  b_{_{//}} = A ( x_p + x_h ) = A x_p$<br>
$\qquad$ the decomposition does not change, we just have too many ways to reach $b_{_{//}}$

$\qquad$ Let's look at an example: $A x = b$, with $A = \left(\begin{array}{ccr} 1 & 3 & -1 \\ 1 & 0 & -2 \\ 0 & 0 & 0 \end{array}\right),\quad b = \left(\begin{array}{r} -2 \\ -1 \\ 3\end{array}\right)$

##### 2.2.1 Implementation

In [6]:
def mk_vec_with_arrow_size_and_colors( A, x, arrow_length, arrow_size, color ):
    '''compute start and endpoints to show successive col_view additions in 3D'''
    start_end_list  = []
    start           = np.zeros( (A.shape[0],1) )

    for i in range( A.shape[1]):
        v               = (x[i]*A[:,i]).reshape(A.shape[0],1)
        if np.linalg.norm(v) > 1.e-5:
            start_end_list.append( [ start.copy().reshape(-1), (start.copy()+v).reshape(-1),  arrow_length, arrow_size, color ] )
        start          += v
    return start_end_list

def multiple_solutions( vecs_with_arrow_sizes_and_colors, v1,v2, x_range=(-10, 10), y_range=(-10, 10), z_range=(-10,10),
                        camera_position=dict(x=0.2, y=0.5, z=.1),
                        arrow_length=1, arrow_size=1, width=500 ):
    traces  = []
    for (start, end, arrow_length, arrow_size, color) in vecs_with_arrow_sizes_and_colors:
        traces +=  vector_trace( start, end, arrow_length, arrow_size, color=color)
    traces += plane_from_basis_vectors_trace( v1, v2, x_range=x_range, y_range=y_range, plane_color='lightgray' )
    return create_fig( traces, x_range=x_range, y_range=y_range, z_range=z_range, camera_position=camera_position, width=width ) 

In [7]:
A  = np.array([[1,3,-1],
               [1,0,-2]])

# Given matrix A column rank < N, find a right hand side and two different solutions
x1 = np.array([[3,-1,2]]).T                                                       # solution of A x = b_parallel
b  = A @ x1                                                                       # b_parallel
# -------------------------------------------------------------------------------------------------------------
np.random.seed(13413)
ht = 3.
xh = 5*(np.eye(A.shape[1]) -  np.linalg.pinv(A)@A) @np.random.rand( A.shape[1])   # a homogeneous solution
x2 = x1-xh.reshape( A.shape[1],1 )                                                # another solution of A x = b_parallel

In [8]:
# Plot the vector b vector and it's orthogonal projection onto b_parallel in the column space of A
#     together with column view additions of A x = b_parallel for two different solutions

def show_plot():
    return \
pn.Column( pn.pane.Markdown("# Two Solutions of A x = b_parallel, the Orthogonal Projection of a Vector b"),
    pn.Row( pn.pane.Plotly( multiple_solutions( mk_vec_with_arrow_size_and_colors( A, x1, .2,1, "green")+\
                                    mk_vec_with_arrow_size_and_colors( A, x2, .2,1, "blue")+\
                                  [[np.array([0,0,0]),np.array([b[0,0],b[1,0],ht]), .2,1,"red"],
                                   [np.array([0,0,0]),np.array([b[0,0],b[1,0],0]),  .3,1, "darkred"],
                                   [np.array([b[0,0],b[1,0],0]), np.array([b[0,0],b[1,0],ht]), .2,1, "darkred"],
], np.array( [A[0,0], A[1,0], 0]), np.array( [A[0,1], A[1,1], 0]))),

    pn.Spacer(width=20),
    pn.Column(
    pn.pane.Markdown(r"### A red vector and its orthogonal decomposition<br> into a vector in a plane a vector orthogonal to the plane", width=500),
    pn.pane.LaTeX(r"The decomposition of  $b = b_{_{//}} + b_{_\perp}$ is shown in dark red", width=500),
    pn.pane.LaTeX(r"The blue and the green path show two solutions of $A x = b_{_{//}}$", width=500)
    )
))

##### 2.2.2 Display

In [9]:
show_plot()

Column
    [0] Markdown(str)
    [1] Row
        [0] Plotly(Figure)
        [1] Spacer(width=20)
        [2] Column
            [0] Markdown(str, width=500)
            [1] LaTeX(str, width=500)
            [2] LaTeX(str, width=500)

**We don't need an infinite number of solutions, just one!**<br>
$\qquad$ The issue is that we have too many vectors in the plane: the columns of $A$ are not linearly independent!

**Solution:** Instead of all columns of $A$, use a basis for the column space $\mathscr{C}(A)$, making the solution unique.<br>
$\qquad$ The new matrix $A$ then has full column rank, and $A^t A$ will be invertible.

## 2.3 Reduced Problem

Instead of solving $\;\; A x = b$, with $A = \begin{pmatrix} 1 & 3 & -1 \\ 1 & 0 & -2 \\0 & 0 & 0\end{pmatrix},\quad b = \left(\begin{array}{r} -2 \\ -1 \\ 3\end{array}\right)$,<br>
$\qquad$ we can remove non-pivot columns of $A$, and use $\tilde{A} = \left(\begin{array}{ccr} 1 & 3 \\ 1 & 0 \\ 0 & 0 \end{array}\right)$ to decompose $b$.<br>
$\qquad$ since $\mathscr{C}(A) = \mathscr{C}(\tilde{A})$, we will obtain the same decomposition.

____
It is now simple to obtain the projection matrices (note we avoid computing the inverse):

In [8]:
A_tilde    = np.array( [[1,3.],[1,0],[0,0]])

#P_parallel = np.round(A_tilde @ np.linalg.inv(A_tilde.T @ A_tilde) @ A_tilde.T)
P_parallel = np.round( A_tilde @ np.linalg.solve( A_tilde.T @ A_tilde, A_tilde.T))
print("Orthogonal Projection Matrix onto the column space of A")
sp.Matrix(P_parallel)

Orthogonal Projection Matrix onto the column space of A


Matrix([
[1.0,   0, 0],
[  0, 1.0, 0],
[  0,   0, 0]])

**Remark:** Since the dimension of $\mathscr{N}(A)$ is equal to 1,<br>
$\qquad$ computing $P_{_{//}} = I - P_{_\perp}$ may be simpler, since the required inverse reduces to a scalar.

$\qquad$ For our example, an orthogonal basis vector for $\mathscr{N}(A^t) = \mathscr{N}(\tilde{A}^t)\;\;$ is $\;\; p = \begin{pmatrix} 0\\ 0 \\ 1 \end{pmatrix}$, so that

$\qquad P_{_\perp} = p (p^tp)^{-1} p^t = p p^t = \begin{pmatrix} 0 & 0 & 0 \\  0 & 0 & 0 \\  0 & 0 & 1 \end{pmatrix}$.